Import packages

In [33]:
import pandas as pd
import numpy as np  
import re #for datacleaning
import nltk  
nltk.download('stopwords')  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score #for measuring accuracy in text category models
from sklearn.model_selection import train_test_split #to divide in train and test set
from sklearn.feature_extraction.text import TfidfVectorizer  #for tfidf vectorize
from sklearn import naive_bayes, svm #for text categorization models
from sklearn.feature_extraction.text import CountVectorizer #for vectorizing
from sklearn.decomposition import LatentDirichletAllocation #for the topic mining model

[nltk_data] Downloading package stopwords to /Users/Amy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The packages <b> langdetect </b> and <b> pyLDAvis </b> need to be downloaded and installed using pip. 

In [2]:
from langdetect import detect
import pyLDAvis.sklearn

# Text Categorization

Import datasets. The datasets are webscraped from the lists on Goodreads: <br>
for non-fiction: https://www.goodreads.com/list/show/134.Best_Non_Fiction_no_biographies_ <br>
for fiction: https://www.goodreads.com/list/show/2522.Cult_Classics

In [3]:
nf = pd.read_csv('booksSummaryNF.txt', sep="|", header=None)
f = pd.read_csv("booksSummaryCult_Classics.txt",sep="|", header=None)

I rename datasets' columns and add a new column specifing the genre original dataset

In [4]:
f = f.rename(columns={0: "title", 1: "rating",2:"summary"})
nf = nf.rename(columns={0: "title", 1: "rating",2:"summary"})
f["genre"]="fiction"
nf["genre"]="non fiction"

I append the two datasets.

In [5]:
df=f.append(nf,ignore_index = True)

I define the genre of the book (fiction or non-fiction) as the target value.

In [6]:
X=df["summary"]
y=df["genre"]

Perform data cleaning on the dataset.

In [8]:
documents = []
stemmer = WordNetLemmatizer()

for i in range(0, len(X)):  
    document = re.sub(r'\W', ' ', str(X[i])) #remove all the special characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) #remove all single characters
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) #remove single characters from the start
    document = re.sub(r'\s+', ' ', document, flags=re.I)  # substitute multiple spaces with single space
    document = document.lower() #converte to Lowercase
    document = document.split() #lemmatization
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)

I divide the dataset in test and train.

In [9]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(documents,y,test_size=0.3)

I apply TfidfVectorizer

In [10]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(documents)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

<b> Naive Bayes classifier</b> model, first fit the model on the train set then predict the values of the test set and calculate the accuracy:

In [11]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(y_pred=predictions_NB, y_true=Test_Y)*100)

Naive Bayes Accuracy Score ->  84.95726495726495


<b> SVM</b> model, first fit the model on the train set then predict the values of the test set and calculate the accuracy:

In [12]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(y_pred=predictions_SVM,y_true= Test_Y)*100)

SVM Accuracy Score ->  84.61538461538461


# Topic Mining

Import datasets. The datasets are webscraped from the lists on Goodreads: <br>
for non-fiction: https://www.goodreads.com/list/show/134.Best_Non_Fiction_no_biographies_ <br>
for fiction: https://www.goodreads.com/list/show/2522.Cult_Classics <br>
for young adult: https://www.goodreads.com/list/show/43.Best_Young_Adult_Books <br>
for sci-fi: https://www.goodreads.com/list/show/19341.Best_Science_Fiction <br>
for fantasy: https://www.goodreads.com/list/show/50.The_Best_Epic_Fantasy_fiction_ <br>
for thriller: https://www.goodreads.com/list/show/348.Thrillers <br>
for romance: https://www.goodreads.com/list/show/12362.All_Time_Favorite_Romance_Novels

In [34]:
ya= pd.read_csv("booksSummaryYoung_Adult.txt", sep="|",header=None)
f = pd.read_csv("booksSummaryCult_Classics.txt", sep="|", header=None)
nf = pd.read_csv('booksSummaryNF.txt', sep="|", header=None)
rom=pd.read_csv("booksSummaryRomance_Novels.txt", sep="|", header=None)
thr=pd.read_csv("booksSummaryThrillers.txt", sep="|", header=None)
sifi=pd.read_csv("booksSummaryScience_Fiction.txt", sep="|", header=None)
fan=pd.read_csv("booksSummaryEpic_Fantasy.txt", sep="|", header=None)
f = f.rename(columns={0: "title", 1: "rating",2:"summary"})
ya = ya.rename(columns={0: "title", 1: "rating",2:"summary"})
nf = nf.rename(columns={0: "title", 1: "rating",2:"summary"})
rom = rom.rename(columns={0: "title", 1: "rating",2:"summary"})
thr = thr.rename(columns={0: "title", 1: "rating",2:"summary"})
sifi = sifi.rename(columns={0: "title", 1: "rating",2:"summary"})
fan=fan.drop(columns=[0])
fan = fan.rename(columns={1: "title", 2: "rating", 3:"summary"})

I unite the datasets.

In [35]:
fic=f.append(ya,ignore_index = True)
fic=fic.append(nf,ignore_index=True)
fic=fic.append(rom,ignore_index=True)
fic=fic.append(thr,ignore_index=True)
fic=fic.append(sifi,ignore_index=True)
fic=fic.append(fan,ignore_index=True)

For <b>Data Cleaning</b>, I detect the language of each summary using library langdetect

In [36]:
fic["language"]=""
for i in range(0,len(fic)):
    fic.at[i,"language"]=detect(fic.loc[i,"summary"])

I drop the duplicated titles.

In [37]:
fic_clean=fic.drop_duplicates(['title'])

I take only the summaries in English.

In [38]:
fic_final=fic_clean.loc[fic_clean['language'] == "en"]
print("The dataset has ", fic_final.shape[0], "summaries.")

The dataset has  5700 summaries.


I create a vector for the words in the data set and a LDA model, with 6 topics.

In [21]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(fic_final["summary"])
print(dtm_tf.shape)
lda_tf = LatentDirichletAllocation(n_components=6, random_state=0)
lda_tf.fit(dtm_tf)

(5700, 6530)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=5, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

Visualizing the topics

In [19]:
pyLDAvis.enable_notebook()

In [22]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.061872 -0.184822       1        1  29.672994
1      0.159670  0.093630       2        1  22.770171
4     -0.051687  0.065541       3        1  22.364866
3     -0.020599  0.028772       4        1  17.941399
0     -0.149256 -0.003121       5        1   7.250570, topic_info=     Category         Freq       Term        Total  loglift  logprob
3463  Default  1531.000000       love  1531.000000  30.0000  30.0000
3211  Default   965.000000       just   965.000000  29.0000  29.0000
6320  Default  1112.000000        war  1112.000000  28.0000  28.0000
3380  Default  2633.000000       life  2633.000000  27.0000  27.0000
3241  Default   374.000000     killer   374.000000  26.0000  26.0000
2492  Default   538.000000       girl   538.000000  25.0000  25.0000
634   Default  1584.000000       book  1584.000000  24.0000  24.0000
1755  Default   697.000000      earth   697.000000  23.0000  23.0000
3515  Default   526.000000      magic   526.000000  22.0000  22.0000
3415  Default   474.000000     little   474.000000  21.0000  21.0000
218   Default   474.000000    ancient   474.000000  20.0000  20.0000
3802  Default   416.000000     murder   416.000000  19.0000  19.0000
3252  Default   491.000000       king   491.000000  18.0000  18.0000
2757  Default   731.000000    history   731.000000  17.0000  17.0000
1393  Default   627.000000        day   627.000000  16.0000  16.0000
1654  Default   368.000000      doesn   368.000000  15.0000  15.0000
5051  Default   458.000000    science   458.000000  14.0000  14.0000
2821  Default   903.000000      human   903.000000  13.0000  13.0000
808   Default   289.000000       case   289.000000  12.0000  12.0000
4290  Default   233.000000     police   233.000000  11.0000  11.0000
3389  Default   971.000000       like   971.000000  10.0000  10.0000
5856  Default   227.000000   thriller   227.000000   9.0000   9.0000
1989  Default   416.000000       evil   416.000000   8.0000   8.0000
481   Default   373.000000     battle   373.000000   7.0000   7.0000
6513  Default   757.000000       year   757.000000   6.0000   6.0000
5379  Default   260.000000      space   260.000000   5.0000   5.0000
205   Default   543.000000   american   543.000000   4.0000   4.0000
1538  Default   193.000000  detective   193.000000   3.0000   3.0000
6319  Default   409.000000      wants   409.000000   2.0000   2.0000
2218  Default   419.000000    fiction   419.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
5379   Topic5    90.095130      space   260.956610   1.5606  -5.6758
563    Topic5    78.207660        big   213.583935   1.6194  -5.8173
1393   Topic5   154.975348        day   627.048440   1.2263  -5.1334
2515   Topic5    84.950279      going   250.874283   1.5412  -5.7346
1670   Topic5    84.664788        don   250.298186   1.5401  -5.7380
5479   Topic5    62.094988       stay   157.781158   1.6915  -6.0480
2238   Topic5    45.791069       film    99.014188   1.8529  -6.3526
287    Topic5    37.508400       aren    70.479711   1.9933  -6.5521
5826   Topic5    90.293078     things   408.188133   1.1154  -5.6736
5825   Topic5    80.971586      thing   331.022153   1.2160  -5.7826
3756   Topic5    61.578373      money   194.130340   1.4759  -6.0564
3266   Topic5   104.544325       know   579.212922   0.9120  -5.5271
3389   Topic5   130.738312       like   971.007460   0.6190  -5.3035
6513   Topic5   113.233436       year   757.008350   0.7242  -5.4472
3122   Topic5    66.420216        isn   242.153730   1.3305  -5.9807
2480   Topic5    55.120502       gets   158.596651   1.5672  -6.1672
3380   Topic5   164.144238       life  2633.382256  -0.1512  -5.0759
3879   Topic5   146.910781        new  2338.764152  -0.1435  -5.1869
3532   Topic5    93.079001       make   685.652729   0.6272  -5.6432
2821   Topic5   101.440269      human   903.661319   0.4371 